In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
df=pd.read_csv('grant_data_imb.csv')

In [5]:
df.sample()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
653,0,4D,10A,NaN,321015.0,100.0,0.0,0.0,0.0,0.0,...,2713.0,25.0,NaN,>10 to 15,1.0,1.0,6.0,3.0,6.0,2.0


In [4]:
pd.set_option('display.max_columns',None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Grant.Status                            4113 non-null   int64  
 1   Sponsor.Code                            3856 non-null   object 
 2   Grant.Category.Code                     3856 non-null   object 
 3   Contract.Value.Band...see.note.A        1953 non-null   object 
 4   RFCD.Code.1                             3853 non-null   float64
 5   RFCD.Percentage.1                       3853 non-null   float64
 6   RFCD.Code.2                             3853 non-null   float64
 7   RFCD.Percentage.2                       3853 non-null   float64
 8   RFCD.Code.3                             3853 non-null   float64
 9   RFCD.Percentage.3                       3853 non-null   float64
 10  RFCD.Code.4                             3853 non-null   floa

In [6]:
target = df['Grant.Status']
features = df.drop(['Grant.Status'], axis=1)

In [7]:
target.value_counts()

Grant.Status
0    3259
1     854
Name: count, dtype: int64

In [8]:
def upsample(features, target, repaet=10):
    features_zeros=features[target==0]
    features_ones=features[target==1]
    target_zeros=target[target==0]
    target_ones=target[target==1]
    if len(target_ones)>len(target_zeros):
        repaet=round(len(target_ones)/len(target_zeros))
        features_upsampled=pd.concat([features_ones]+ [features_zeros]*repaet)
        target_upsampled=pd.concat([target_ones]+[target_zeros]*repaet)
    else:
        repaet=round(len(target_zeros)/len(target_ones))
        features_upsampled=pd.concat([features_zeros]+ [features_ones]*repaet)
        target_upsampled=pd.concat([target_zeros]+[target_ones]*repaet)
        
    features_upsampled, target_upsampled=shuffle(features_upsampled, target_upsampled, random_state=23)
    return features_upsampled, target_upsampled


In [9]:
features_train_upsampled, target_train_upsampled=upsample(features, target)

In [10]:
target_train_upsampled.value_counts()

Grant.Status
1    3416
0    3259
Name: count, dtype: int64

In [17]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            3856 non-null   object 
 1   Grant.Category.Code                     3856 non-null   object 
 2   Contract.Value.Band...see.note.A        1953 non-null   object 
 3   RFCD.Code.1                             3853 non-null   float64
 4   RFCD.Percentage.1                       3853 non-null   float64
 5   RFCD.Code.2                             3853 non-null   float64
 6   RFCD.Percentage.2                       3853 non-null   float64
 7   RFCD.Code.3                             3853 non-null   float64
 8   RFCD.Percentage.3                       3853 non-null   float64
 9   RFCD.Code.4                             3853 non-null   float64
 10  RFCD.Percentage.4                       3853 non-null   floa

In [11]:
for col in features.columns:
    if (features[col].dtype=='object'):
        print(col)
        print(features[col].value_counts())
        print('--------------')

Sponsor.Code
Sponsor.Code
4D      1006
2B       915
21A      375
24D      114
40D       91
        ... 
199C       1
313A       1
373A       1
298B       1
225A       1
Name: count, Length: 226, dtype: int64
--------------
Grant.Category.Code
Grant.Category.Code
10A    2050
30B     707
50A     375
10B     211
20C     180
30C     147
30D      93
20A      49
30G      35
30E       5
30A       2
40C       1
30F       1
Name: count, dtype: int64
--------------
Contract.Value.Band...see.note.A
Contract.Value.Band...see.note.A
A     961
B     305
C     159
D     151
G     135
E      98
F      75
H      33
J      18
I      11
P       2
K       1
M       1
O       1
Q       1
L       1
Name: count, dtype: int64
--------------
Role.1
Role.1
CHIEF_INVESTIGATOR         3640
EXT_CHIEF_INVESTIGATOR      230
PRINCIPAL_SUPERVISOR        144
DELEGATED_RESEARCHER         36
STUD_CHIEF_INVESTIGATOR      10
HONVISIT                      6
EXTERNAL_ADVISOR              1
Name: count, dtype: int64
---------

In [13]:
features['With.PHD.1'].fillna('No', inplace=True)
for col in features.columns:
    if (features[col].dtype=='object'):
        features[col].fillna('not indicated', inplace=True)

In [21]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            4113 non-null   object 
 1   Grant.Category.Code                     4113 non-null   object 
 2   Contract.Value.Band...see.note.A        4113 non-null   object 
 3   RFCD.Code.1                             3853 non-null   float64
 4   RFCD.Percentage.1                       3853 non-null   float64
 5   RFCD.Code.2                             3853 non-null   float64
 6   RFCD.Percentage.2                       3853 non-null   float64
 7   RFCD.Code.3                             3853 non-null   float64
 8   RFCD.Percentage.3                       3853 non-null   float64
 9   RFCD.Code.4                             3853 non-null   float64
 10  RFCD.Percentage.4                       3853 non-null   floa

In [14]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            4113 non-null   object 
 1   Grant.Category.Code                     4113 non-null   object 
 2   Contract.Value.Band...see.note.A        4113 non-null   object 
 3   RFCD.Code.1                             3853 non-null   float64
 4   RFCD.Percentage.1                       3853 non-null   float64
 5   RFCD.Code.2                             3853 non-null   float64
 6   RFCD.Percentage.2                       3853 non-null   float64
 7   RFCD.Code.3                             3853 non-null   float64
 8   RFCD.Percentage.3                       3853 non-null   float64
 9   RFCD.Code.4                             3853 non-null   float64
 10  RFCD.Percentage.4                       3853 non-null   floa

In [15]:
features_one=pd.get_dummies(features, drop_first=True)
features_one.sample()

,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,...,Country.of.Birth.1_Western Europe,Country.of.Birth.1_not indicated,Home.Language.1_Other,Home.Language.1_not indicated,With.PHD.1_Yes,No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,No..of.Years.in.Uni.at.Time.of.Grant.1_more than 15,No..of.Years.in.Uni.at.Time.of.Grant.1_not indicated
1495,321008.0,50.0,321019.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,True,False,True,False,False,True,False,False,False


In [16]:
features_train, features_test, target_train, target_test = train_test_split(features_one, target, test_size=0.25, random_state=23)


In [18]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

In [19]:
pd.DataFrame(features_test_scaled,columns=features_train.columns)

,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,...,Country.of.Birth.1_Western Europe,Country.of.Birth.1_not indicated,Home.Language.1_Other,Home.Language.1_not indicated,With.PHD.1_Yes,No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,No..of.Years.in.Uni.at.Time.of.Grant.1_more than 15,No..of.Years.in.Uni.at.Time.of.Grant.1_not indicated
0,0.128656,-0.558664,1.004788,0.129394,1.548264,1.101646,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,-0.448779,-0.139657,0.324007,0.896975,-0.461238,-0.694248,-0.370928,3.170767,-0.484343
1,0.120376,-0.931331,1.001654,1.682557,-0.639349,-0.587109,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,2.228269,-0.139657,0.324007,-1.114858,-0.461238,-0.694248,-0.370928,-0.315381,2.064655
2,0.122405,0.932004,-0.988570,-0.906048,-0.639349,-0.587109,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,-0.448779,-0.139657,-3.086357,0.896975,-0.461238,1.440408,-0.370928,-0.315381,-0.484343
3,-1.523448,-0.558664,0.502083,0.647115,0.997261,0.257269,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,-0.448779,-0.139657,0.324007,0.896975,-0.461238,1.440408,-0.370928,-0.315381,-0.484343
4,-0.469897,-0.931331,0.822305,0.647115,1.340505,1.101646,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,-0.448779,-0.139657,0.324007,0.896975,-0.461238,1.440408,-0.370928,-0.315381,-0.484343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,0.132776,0.932004,-0.988570,-0.906048,-0.639349,-0.587109,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,-0.448779,-0.139657,0.324007,0.896975,2.168076,-0.694248,-0.370928,-0.315381,-0.484343
1025,-0.295658,0.932004,-0.988570,-0.906048,-0.639349,-0.587109,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,-0.448779,-0.139657,0.324007,0.896975,-0.461238,-0.694248,-0.370928,-0.315381,2.064655
1026,1.749852,-1.303999,1.681183,0.647115,2.222615,1.946024,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,2.228269,-0.139657,0.324007,-1.114858,-0.461238,-0.694248,-0.370928,-0.315381,2.064655
1027,0.112157,0.932004,-0.988570,-0.906048,-0.639349,-0.587109,-0.146968,-0.13478,-0.066265,-0.062406,...,-0.197696,-0.448779,-0.139657,0.324007,0.896975,-0.461238,1.440408,-0.370928,-0.315381,-0.484343


In [21]:
model=LogisticRegressionCV(solver='liblinear',random_state=12,class_weight='balanced',cv=10)
model.fit(features_train_scaled, target_train)
roc_auc_score(target_test, model.predict_proba(features_test_scaled)[:,1])

ValueError: Input X contains NaN.
LogisticRegressionCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values